In [1]:
import numpy as np
import pandas as pd
import copy
import mohusets.fuzzynumbers as fns
import mohusets.fuzzysets as fs
import mohusets.aggregators as agg

from utils import maxdev, ranking_dm, show_result

In [2]:
dataFile = 'A.csv'
dataPath = 'data/'+dataFile

lam,tao = 1,1               # Generalized hesitant Fermatean fuzzy distance parameter, lam=1 means Hamming distance, lam=2 means Euclidean distance.
                            # tao represents the standardized risk coefficient, the value [0,1]

qrung = 3

In [3]:
t = fs.fuzzyset(qrung,'qrungdhfe')
t.loadcsv(dataPath)

weight = [0.2,0.15,0.2,0.3,0.15]

In [4]:
t.mat

,0,1,2,3,4
0,"[[0.3, 0.5], [0.6]]","[[0.6], [0.4, 0.6]]","[[0.4], [0.2]]","[[0.5], [0.4]]","[[0.3], [0.6]]"
1,"[[0.8], [0.2]]","[[0.6, 0.7], [0.1]]","[[0.7], [0.3, 0.4]]","[[0.3], [0.2]]","[[0.6], [0.1]]"
2,"[[0.2], [0.3]]","[[0.3], [0.5]]","[[0.5, 0.8], [0.2]]","[[0.3], [0.7]]","[[0.5], [0.4, 0.5]]"


In [5]:
def hadi_madm(ope, decision_matrix, lam, tao, indeterminacy=True, pw=0, reverse=True, **para):
    def _method(ope, alt, w, pw=0, rev=True, **para):
        A = ranking_dm(ope, alt, w, rev, **para)
        score_list = []

        if pw == 0:
            posw = np.array([1 / len(alt[0]) for i in range(len(alt[0]))])
        else:
            posw = pw

        for at in A.set:
            score_list.append(ope(at, posw, **para).score)
        return np.asarray(score_list)

    dm_max = fs.dh_fn_sets(decision_matrix, norm='max')
    dm_min = fs.dh_fn_sets(decision_matrix, norm='min')
    dm_mean = fs.dh_fn_sets(decision_matrix, norm='mean')
    dm = np.array([dm_max, dm_min, dm_mean])
    # weight = maxdev(decision_matrix.set,lam,tao,indeterminacy)
    result = []
    for d in dm:
        result.append(_method(ope, d.set, weight, pw, reverse, **para))
    return result

In [6]:
result1 = hadi_madm(agg.sub_weighted_ave, t, lam=lam, tao=tao, indeterminacy=False)

max_res1 = show_result(t.set,'Hadi\'s method with max',result1[0], reverse=True)
min_res1 = show_result(t.set,'Hadi\'s method with min',result1[1], reverse=True)
mean_res1 = show_result(t.set,'Hadi\'s method with mean',result1[2], reverse=True)
res1 = pd.concat([max_res1, min_res1, mean_res1], axis=0)
res1

,A1,A2,A3,Ranking Order
Hadi's method with max,0.026170,0.300983,0.102053,A2>A3>A1
Hadi's method with min,0.026071,0.276861,0.007010,A2>A1>A3
Hadi's method with mean,0.023222,0.287516,0.037311,A2>A3>A1


In [7]:
result2 = hadi_madm(agg.sub_weighted_geom, t, lam=lam, tao=tao, indeterminacy=False)

max_res2 = show_result(t.set,'Hadi\'s method with max',result2[0], reverse=True)
min_res2 = show_result(t.set,'Hadi\'s method with min',result2[1], reverse=True)
mean_res2 = show_result(t.set,'Hadi\'s method with mean',result2[2], reverse=True)
res2 = pd.concat([max_res2, min_res2, mean_res2], axis=0)
res2

,A1,A2,A3,Ranking Order
Hadi's method with max,-0.058797,0.187068,-0.082743,A2>A1>A3
Hadi's method with min,-0.051779,0.176720,-0.083730,A2>A1>A3
Hadi's method with mean,-0.051140,0.182605,-0.082212,A2>A1>A3
